## Import data

In [1]:
import pickle
with open('splitData.pickle', 'rb') as f:
    texts_train = pickle.load(f)
    texts_val = pickle.load(f)
    texts_test = pickle.load(f)
    labels_train = pickle.load(f)
    labels_val = pickle.load(f)
    labels_test = pickle.load(f)

all_texts  = texts_train  + texts_val  + texts_test
all_labels = labels_train + labels_val + labels_test
# check the ratio of each dataset
total = len(texts_train) + len(texts_val) + len(texts_test)
len(texts_train)/total, len(texts_val)/total, len(texts_test)/total

(0.5997757847533632, 0.20011210762331838, 0.20011210762331838)

# ここからコピーする

In [19]:
# correspond labels(songer names) with integers
label2id = {label: i for i, label in enumerate(set(labels_train))}
id2label = {id:label for label,id in label2id.items()}
id2label

{0: 'Dua Lipa',
 1: 'Taylor Swift',
 2: 'Eminem',
 3: 'Jennifer Lopez',
 4: 'Ariana Grande',
 5: 'Selena Gomez',
 6: 'Michael Jackson',
 7: 'Beyonce Knowles',
 8: 'Lady Gaga',
 9: 'XXXTentacion'}

In [24]:
labels_train_int = [label2id[l] for l in labels_train]
labels_val_int = [label2id[l] for l in labels_val]

In [26]:
from transformers import AutoTokenizer
from datasets import Dataset

tokenizer = AutoTokenizer.from_pretrained("roberta-base",)

def createDataset(texts, labels_int):
    encoded_texts = []
    for t in texts:
        encoded_texts.append(tokenizer.encode(t,truncation=True))
    
    dataset = Dataset.from_dict({
        "input_ids":encoded_texts,
        "labels": labels_int
    })
    return dataset

train_dataset = createDataset(texts_train, labels_train_int)
validation_dataset = createDataset(texts_val, labels_val_int)




ModuleNotFoundError: No module named 'datasets'

In [ ]:
from transformers import set_seed, AutoModelForSequenceClassification

set_seed(42)

model = AutoModelForSequenceClassification.from_pretrained('roberta-base',id2label=id2label)

In [ ]:
# parameters
learning_rate = 1e-4
epochs = 1
batch_size = 16
weight_decay = None

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="singers_model", # HuggingFace wants a name for your model
    evaluation_strategy="epoch", # How often we want to evaluate the model
    learning_rate=learning_rate, # Hyperparameter
    per_device_train_batch_size=batch_size, # Hyperparameter
    per_device_eval_batch_size=batch_size, # Hyperparameter
    num_train_epochs=epochs, # Hyperparameter
    weight_decay=weight_decay, # Hyperparameter
)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


trainer = Trainer(
    model=model, # The model you want to train
    args=training_args, # The various training arguments set up above
    train_dataset=train_dataset, # The data to use to update the weights
    eval_dataset=validation_dataset, # The data to use 
    tokenizer=tokenizer, # The tokenizer used on the data
    data_collator=data_collator, # A data collator that does clever things moving data around
)

In [ ]:
trainer.train()

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from datasets import Dataset

# Load data
data = pd.read_csv("song_lyrics.csv")

# Split data into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(data["lyrics"], data["singer"], test_size=0.2, random_state=42)

# Create PyTorch dataset
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
train_dataset = Dataset.from_dict({'input_ids': tokenizer.batch_encode_plus(train_texts.tolist(), 
                                                                           padding=True, 
                                                                           truncation=True)['input_ids'],
                                   'attention_mask': tokenizer.batch_encode_plus(train_texts.tolist(), 
                                                                                   padding=True, 
                                                                                   truncation=True)['attention_mask'],
                                   'labels': train_labels.tolist()})
test_dataset = Dataset.from_dict({'input_ids': tokenizer.batch_encode_plus(test_texts.tolist(), 
                                                                          padding=True, 
                                                                          truncation=True)['input_ids'],
                                  'attention_mask': tokenizer.batch_encode_plus(test_texts.tolist(), 
                                                                                  padding=True, 
                                                                                  truncation=True)['attention_mask'],
                                  'labels': test_labels.tolist()})

# Convert to PyTorch dataset
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Create data loader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=False)


In [10]:
for i in range(len(texts_train)):
    if "Postcard" in texts_train[i]:
        print(i)

texts_train[634]

634


"Postcard (New Shoes)\nPostcard (New Shoes) Lyrics\nHey, honey\nCat got ya tongue?\nDon't look so funny\nMr. Comedian, joke's on you and\nYou good at bets, so where's my money?\nI know you bet I'd never make that move\nSo, why's the house empty?\nLights off by the end of this week\nI leave wit' what I came wit', baby\nI'll take the dogs, you keep the Mercedes\n'Cause I'm gone, I'm not fazed\nI walk these roads with grace\nAnd I'll wave with you behind me\nWith a new smile on my face, 'cause\n\nI'm walkin' in new shoes now\nI got a new song to sing\nWhen I walk in the room, every head turns\nEvery eye is on me\nToo bad you're not here to see it\nAnd by the time you get this\nI'll be so long, gone and far\nI'll send you a postcard\nI'll send you a postcard\n\nHey, baby\nYou seen a ghost?\nThis ain't yo' lady\nThis is the chick that you held back\nNow, all of the things you said were crazy\nI checked off my list and I feel good about it\nI got the new haircut\nThe one that you said wouldn

In [ ]:
why do I get the error below for the code below?
raise ValueError(
   2972                     f"type of {first_element} unknown: {type(first_element)}. "
   2973                     "Should be one of a python, numpy, pytorch or tensorflow object."
-----------
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
from transformers import set_seed, AutoModelForSequenceClassification

set_seed(42)

model = AutoModelForSequenceClassification.from_pretrained('roberta-base')
# parameters
learning_rate = 1e-4
epochs = 1
batch_size = 16
weight_decay = None


from datasets import Dataset
import numpy as np


train_dataset = Dataset.from_dict({"input_ids":texts_train,"labels":labels_train})
validation_dataset = Dataset.from_dict({"input_ids":texts_val, "labels":labels_val})
train_dataset[0]

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="singers_model", # HuggingFace wants a name for your model
    evaluation_strategy="epoch", # How often we want to evaluate the model
    learning_rate=learning_rate, # Hyperparameter
    per_device_train_batch_size=batch_size, # Hyperparameter
    per_device_eval_batch_size=batch_size, # Hyperparameter
    num_train_epochs=epochs, # Hyperparameter
    weight_decay=weight_decay, # Hyperparameter
)


from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


trainer = Trainer(
    model=model, # The model you want to train
    args=training_args, # The various training arguments set up above
    train_dataset=train_dataset, # The data to use to update the weights
    eval_dataset=validation_dataset, # The data to use 
    tokenizer=tokenizer, # The tokenizer used on the data
    data_collator=data_collator, # A data collator that does clever things moving data around
)

trainer.train()


